In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./data/mnist", one_hot = True)

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [3]:
x_true = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, [None, 10])

In [4]:
weight_1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev = 0.01))
hidden_1 = tf.nn.conv2d(x_true, weight_1, strides = [1, 1, 1, 1], padding = "SAME")
hidden_1 = tf.nn.relu(hidden_1)
hidden_1 = tf.nn.max_pool(hidden_1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")
hidden_1 = tf.nn.dropout(hidden_1, keep_prob = 0.8)

In [5]:
weight_2 = tf.Variable(tf.truncated_normal([3, 3, 32, 64], stddev = 0.01))
hidden_2 = tf.nn.conv2d(hidden_1, weight_2, strides = [1, 1, 1, 1], padding = "SAME")
hidden_2 = tf.nn.relu(hidden_2)
hidden_2 = tf.nn.max_pool(hidden_2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

In [6]:
hidden_2 = tf.reshape(hidden_2, [-1, 7 * 7 * 64])
hidden_2 = tf.nn.dropout(hidden_2, keep_prob = 0.8)

In [7]:
weight_3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 256], stddev = 0.01))
fc_1 = tf.matmul(hidden_2, weight_3)
fc_1 = tf.nn.relu(fc_1)
fc_1 = tf.nn.dropout(fc_1, 0.5)

In [8]:
weight_4 = tf.Variable(tf.truncated_normal([256, 10], stddev = 0.01))
y_pred = tf.matmul(fc_1, weight_4)

In [9]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_true)
cost = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimizer = optimizer.minimize(cost)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 100
num_batch = int(mnist.train.num_examples / batch_size)

In [12]:
for epoch in range(10):
    total_cost = 0
    for i in range(num_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_value = sess.run([optimizer, cost], {x_true : batch_xs, y_true : batch_ys})
        total_cost += cost_value
    print("Epoch : {0}, Cost : {1}".format(epoch + 1, total_cost / num_batch))
print("Done")

Epoch : 1, Cost : 0.23439359572800722
Epoch : 2, Cost : 0.1320878180116415
Epoch : 3, Cost : 0.12076560272750529
Epoch : 4, Cost : 0.11472843578000638
Epoch : 5, Cost : 0.11035569337111982
Epoch : 6, Cost : 0.1038506305039945
Epoch : 7, Cost : 0.1074991184938699
Epoch : 8, Cost : 0.10531274641271342
Epoch : 9, Cost : 0.11388580821369859
Epoch : 10, Cost : 0.10443375584796409
Done


In [14]:
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy : {0}".format(sess.run(accuracy, {x_true : mnist.test.images.reshape(-1, 28, 28, 1), y_true : mnist.test.labels})))

Accuracy : 0.9699000120162964
